In [5]:
import pandas as pd
from pathlib import Path
import datetime

def predict_row(row, total_games, correct_predictions, prediction_list, hca = 0):
    date = row['game_date']
    home_team = row['team_name_home']
    away_team = row['team_name_away']
    total_games += 1
    home_total_games = row['record_home_wins'] + row['record_home_losses']
    away_total_games = row['record_away_wins'] + row['record_away_losses']
    home_win_pct = 0
    away_win_pct = 0
    if home_total_games == 0 and away_total_games == 0:
        if row['wl_home'] == 'W':
            correct_predictions += 1
    else :
        if home_total_games == 0:
            home_win_pct = 0.5
        else:
            home_win_pct = float(row['record_home_wins'])/float(home_total_games)
        if away_total_games == 0:
            away_win_pct = 0.5
        else:
            away_win_pct = float(row['record_away_wins'])/float(away_total_games)
        if home_win_pct + hca >= away_win_pct and row['wl_home'] == 'W':
            correct_predictions += 1
        elif (away_win_pct > home_win_pct + hca) and row['wl_home'] == 'L':
            correct_predictions += 1
    if home_win_pct + hca >= away_win_pct:
        prediction_list.append(str(date) + ": " + str(home_team) + "-" + str(away_team) + ", predicted winner:" + str(home_team))
    else: prediction_list.append(str(date) + ": " + str(home_team) + "-" + str(away_team) + ", predicted winner:" + str(away_team))
    return total_games, correct_predictions, prediction_list

In [18]:
import pandas as pd
from pathlib import Path
import datetime

def add_predictions(games_table, hca = 0):
    games_table['home_total_games'] = games_table['record_home_wins'] + games_table['record_home_losses'] - 1
    games_table['away_total_games'] = games_table['record_away_wins'] + games_table['record_away_losses'] - 1
    games_table['home_win_pct'] = 0.5
    games_table['away_win_pct'] = 0.5
    games_table['will_home_win'] = 'L'
    games_table.loc[(games_table['home_total_games'] > 0) & (games_table['wl_home'] == 'W'), 'home_win_pct'] = (games_table['record_home_wins'].astype(float) - 1)/games_table['home_total_games'].astype(float)
    games_table.loc[(games_table['home_total_games'] > 0) & (games_table['wl_home'] == 'L'), 'home_win_pct'] = games_table['record_home_wins'].astype(float)/games_table['home_total_games'].astype(float)
    games_table.loc[(games_table['away_total_games'] > 0) & (games_table['wl_home'] == 'W'), 'away_win_pct'] = games_table['record_away_wins'].astype(float)/games_table['away_total_games'].astype(float)
    games_table.loc[(games_table['away_total_games'] > 0) & (games_table['wl_home'] == 'L'), 'away_win_pct'] = (games_table['record_away_wins'].astype(float) - 1)/games_table['away_total_games'].astype(float)
    games_table.loc[games_table['home_win_pct'] + hca >= games_table['away_win_pct'], 'will_home_win'] = 'W'
    return games_table

In [19]:
accuracy = 0
total_games = 0
correct_predictions = 0

#correct_prediction_list = []
base_dir = Path.cwd()
games = pd.read_csv(base_dir / "all_games.csv")
games['game_date'] = pd.to_datetime(games['game_date'])
games_train = games[(games['game_date'] <= datetime.datetime(2023,9,30))]
games_test = games[(games['game_date'] >= datetime.datetime(2023,10,1)) & (games['game_date'] <= datetime.datetime(2024,9,30))]
games_new = games[games['game_date'] >= datetime.datetime(2024,10,1)]
#hca_values = [0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2]
#for hca in hca_values:
#    for idx, row in games_train.iterrows():
#        total_games, correct_predictions, prediction_list = predict_row(row, total_games, correct_predictions, prediction_list, hca)
#    correct_prediction_list.append(correct_predictions)
#    correct_predictions = 0
#print(correct_prediction_list)
#The best hca parameter consistently turns out to be 0.06
hca = 0.06
games_train = add_predictions(games_train, hca)
total_games = len(games_train)
correct_predictions = len(games_train[games_train['wl_home'] == games_train['will_home_win']])
accuracy = float(correct_predictions)/float(total_games)
print("Training: " + str(correct_predictions) + "/" + str(total_games) + ", " + str(accuracy))
games_train.to_csv(base_dir / "games_trained.csv")

games_test = add_predictions(games_test, hca)
total_games = len(games_test)
correct_predictions = len(games_test[games_test['wl_home'] == games_test['will_home_win']])
accuracy = float(correct_predictions)/float(total_games)
print("Testing: " + str(correct_predictions) + "/" + str(total_games) + ", " + str(accuracy))
games_test.to_csv(base_dir / "games_tested.csv")

games_new = add_predictions(games_new, hca)
total_games = len(games_new)
correct_predictions = len(games_new[games_new['wl_home'] == games_new['will_home_win']])
accuracy = float(correct_predictions)/float(total_games)
print("New season: " + str(correct_predictions) + "/" + str(total_games) + ", " + str(accuracy))
games_new.to_csv(base_dir / "games_tested.csv")

C:\Users\User\AppData\Local\Temp\ipykernel_14800\3235406598.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_table['home_total_games'] = games_table['record_home_wins'] + games_table['record_home_losses'] - 1
C:\Users\User\AppData\Local\Temp\ipykernel_14800\3235406598.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_table['away_total_games'] = games_table['record_away_wins'] + games_table['record_away_losses'] - 1
C:\Users\User\AppData\Local\Temp\ipykernel_14800\3235406598.py:8: SettingWit

Training: 45309/69192, 0.6548300381547
Testing: 851/1319, 0.645185746777862
New season: 233/389, 0.5989717223650386


C:\Users\User\AppData\Local\Temp\ipykernel_14800\3235406598.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_table['home_total_games'] = games_table['record_home_wins'] + games_table['record_home_losses'] - 1
C:\Users\User\AppData\Local\Temp\ipykernel_14800\3235406598.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_table['away_total_games'] = games_table['record_away_wins'] + games_table['record_away_losses'] - 1
C:\Users\User\AppData\Local\Temp\ipykernel_14800\3235406598.py:8: SettingWit